# Ollama

In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,32

In [2]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [3]:
# !ollama pull deepseek-r1:1.5b  # or your preferred model
# !ollama pull nomic-embed-text #
# !ollama pull llama3.2:3b
!ollama pull gpt-oss:20b
# !ollama pull paraphrase-multilingual
# !ollama pull gemma3:12b


# Installing more package

In [4]:
# Clone the repository if it doesn't exist
# !git clone https://github.com/daoanhkhoa123/SupertCagRag
# %cd /kaggle/working/SupertCagRag

# Install required packages from requirements.txt
!pip install -q langchain langchain-ollama ollama
! pip install -q tqdm
# Update system packages and install additional tools
# !apt-get update -q
# !apt-get install -y -q poppler-utils

# !pip install -U -q langchain-community
# !pip install -qU transformers

# !pip install -q git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn optimum auto-gptq bitsandbytes
# !pip install -q ollama
# !pip install -q colab-xterm
# !sudo apt-get update
# !sudo apt-get install poppler-utils

In [5]:
import os
os.environ['PATH'] = '/usr/bin:' + os.environ['PATH']
!which pdfinfo
!pdfinfo -v

/bin/bash: line 1: pdfinfo: command not found


 # Code

In [6]:
TEMPLATE = """
You are given:
- A student's subject scores (0–1000 scale) with these interpretations and answer guidelines:

  < 350: Very weak understanding
    → Use simple, everyday language and short sentences.
    → Avoid jargon; if unavoidable, define it simply.
    → Focus on core concepts only; give concrete examples.
    → Avoid technical depth; keep explanations step-by-step.

  350–500: Incomplete understanding
    → Use mostly simple language, introducing basic technical terms with definitions.
    → Explain the main concepts and relationships without overwhelming detail.
    → Use analogies and short examples.
    → Avoid advanced formulas or edge cases.

  500–650: Average understanding
    → Use standard technical vocabulary but explain less common terms.
    → Include moderate detail and simple examples.
    → Focus on solidifying fundamentals while gently introducing intermediate ideas.
    → Keep explanations structured and concise.

  650–800: Good understanding
    → Use precise technical terminology without over-explaining common concepts.
    → Include intermediate to advanced details and discuss key nuances.
    → Use examples and small real-world applications.
    → Briefly touch on complexities and edge cases.

  800–930: Strong understanding
    → Use advanced technical vocabulary and concise explanations.
    → Dive into deeper reasoning, implications, and trade-offs.
    → Use fewer examples unless illustrating complex points.
    → Highlight connections to broader concepts or related topics.

  930–1000: Excellent understanding
    → Use expert-level technical detail and terminology.
    → Discuss advanced nuances, optimizations, and theoretical aspects.
    → Avoid over-explaining basics; focus on high-level insights and subtle points.
    → Include references to related advanced concepts or frameworks.

- A technical document containing the reference content.
- A question to answer.

Your task:
- Tailor the answer according to the student’s score range rules above.
- Focus on concepts the student is most likely to find challenging for their level.
- Ensure accuracy and base reasoning solely on the provided document context.
- Do not explain your reasoning process.
- Keep the answer under 300 tokens.

Inputs:
- Document Context: {context}
- Student Score Profile: {profile}
- Question: {question}

Output:
- The pedagogically adapted answer.
"""

In [7]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
import json

# Load model
model_name = "gpt-oss:20b"
model = OllamaLLM(model=model_name)


# Build the prompt from template
prompt = ChatPromptTemplate.from_template(TEMPLATE)

# Create the chain
chain = prompt | model

# Function to run the chain
def answer_question(context, question, profile):
  return chain.invoke({"question": question, "context": context, "profile":profile})

In [8]:
# answer_question("a", "b" ,"c")

# Run


In [11]:
import pandas as pd
ref = pd.read_csv("/content/100_userquestion2.csv")
# row = ref.copy()
# row

In [12]:
# M ĐỔI THÀNH HEAD(50) Ở ĐÂY

df = ref.tail(50)
# Create chunks of 10 rows each from the last 50 rows
chunks = [df[i:i+10] for i in range(0, len(df), 10)]

df = chunks[0]
df.shape

(10, 6)

In [13]:
from tqdm import tqdm
import json

graded_results = []

# Loop through all entries in DataFrames
for idx, row in enumerate(tqdm(df.iterrows(), total=len(df), desc="Grading Responses")):
    row = row[1]

    try:
        res = answer_question(row["snippet"], row["question"], row["modules"])
    except Exception as e:
        print(f"❌ Error at row {idx}: {e}")
        continue

    if res is None:
        continue

    graded_results.append({
        "userid": row['userid'],
        "profile": row['modules'],
        "context": row['snippet'],
        "question": row['question'],
        "answer": res,
    })

    # Save every 25 entries
    if (idx + 1) % 10 == 0:
        temp_filename = f"dataset_batch_{idx+1}.json"
        with open(temp_filename, "w", encoding="utf-8") as f:
            json.dump(graded_results, f, ensure_ascii=False, indent=2)
        print(f"✅ Saved batch to '{temp_filename}'")

# Final save with full dataset
with open("gptoss_genanswers.json", "w", encoding="utf-8") as f:
    json.dump(graded_results, f, ensure_ascii=False, indent=2)

print("🎉 All responses graded and saved to 'gptoss_genanswers.json'")


Grading Responses: 100%|██████████| 10/10 [04:48<00:00, 28.89s/it]

✅ Saved batch to 'dataset_batch_10.json'
🎉 All responses graded and saved to 'gptoss_genanswers.json'


In [14]:
import pandas as pd

df_graded_results = pd.DataFrame(graded_results)
df_graded_results.to_csv("gptoss_genanswers0.csv", index=False)